In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
import pytz

from scipy.optimize import curve_fit
from scipy.interpolate import interp1d

import gm2
import trfp
import plotting_functions as plt2
import analysis_helper as helper
import helper_function_candidates as helper_old

file_dict = {'1a':['gold_subruns/gold_subruns_60Hr.txt', 'hdf5/2020-09-30_run_1a.h5', '/data2/aetb/2020-09-30_hybrid_maps.h5'],
             '1b':['gold_subruns/gold_subruns_HighKick.txt', 'hdf5/2020-09-30_run_1b.h5', '/data2/aetb/2020-09-30_hybrid_maps.h5'],
             '1c':['gold_subruns/gold_subruns_9day.txt', 'hdf5/2020-09-30_run_1c.h5', '/data2/aetb/2020-09-30_hybrid_maps.h5'],
             '1d':['gold_subruns/gold_subruns_endgame.txt', 'hdf5/2020-09-30_run_1d.h5', '/data2/aetb/2020-09-30_hybrid_maps.h5']}

key_dict = {'1a':{'1':['tr_df_1', 'tr_df_2', 'vtm_1']},
            '1b':{'1':['tr_df_1', 'tr_df_2', 'vtm_1'],
                  '2':['tr_df_2', 'tr_df_3', 'vtm_2']},
            '1c':{'1':['tr_df_1', 'tr_df_2', 'vtm_1'],
                  '2':['tr_df_3', 'tr_df_4', 'vtm_2'],
                  '3':['tr_df_4', 'tr_df_5', 'vtm_3'],
                  '4':['tr_df_5', 'tr_df_6', 'vtm_4']},
            '1d':{'2':['tr_df_2', 'tr_df_3', 'vtm_2'],
                  '3':['tr_df_4', 'tr_df_5', 'vtm_3'],
                  '4':['tr_df_6', 'tr_df_7', 'vtm_4'],
                  '5':['tr_df_7', 'tr_df_8', 'vtm_5'],
                  '6':['tr_df_8', 'tr_df_9', 'vtm_6']}}

def cov(t1, t2, T, M=0.01810259**2):  # BB, units for M are Hz^2/sec
    u = np.minimum(t1, t2)
    v = np.maximum(t1, t2)
    return M * (T - v) * u / T

def cov2(t1, t2, T, M=0.01810259**2):  # RW, units for M are Hz^2/sec
    u = np.minimum(t1, t2)
    v = np.maximum(t1, t2)
    return M * u

# rates = np.array([4.56668293e-03, 2.39702431e-04, 2.23624253e-04,
#          1.06138686e-04, 3.40662192e-04, 3.02648014e-05,
#          7.69850642e-05, 4.63122653e-05, 9.34178986e-05])/np.sqrt(72)  # 12 Oct 2020, `step_2_sync_offsets.ipynb`

# rates = np.array([2.28331841e-03, 1.19851017e-04, 1.11812552e-04, 5.30712951e-05,
#                   1.70331133e-04, 1.51323533e-05, 3.84927434e-05, 2.31560139e-05,
#                   4.67086004e-05])/72  # 12 Jan 2020, `step_2_sync_offsets.ipynb` after fixing Gaussian definition

# rates = np.array([1.17615626e-04, 3.89509569e-06, 2.03150011e-06, 9.59640992e-07,
#                   1.13004927e-05, 2.93817458e-07, 6.89674412e-07, 5.13120763e-07,
#                   8.01024156e-07])  # 13 Jan 2020, `step_2_sync_offsets.ipynb` using 68th quantile def on rates^2
#                                     # Also uses simple correlation estimator
    
rates = np.array([0.0002440148957033999, 7.174497059079315e-06, 5.597953677893461e-06,
                  2.471179896215746e-06, 2.469063223225018e-05, 4.98031484960896e-07,
                  2.3557763230871967e-06, 1.3267701986068543e-06, 2.733157063334935e-06])
                  ## 27 Jan, `sync_offset_studies`, uses station-by-station widths and correlation matrix


Welcome to JupyROOT 6.22/02


In [2]:
run_list = ['1a1', '1b1', '1b2', '1c1', '1c2', '1c3', '1d2', '1d3', '1d4', '1d5', '1d6']
# run_list = ['1a1']

def cov(t1, t2, T, M=0.01810259**2):  # BB, units for M are Hz^2/sec
    u = np.minimum(t1, t2)
    v = np.maximum(t1, t2)
    return M * (T - v) * u / T

def cov2(t1, t2, T, M=0.01810259**2):  # RW, units for M are Hz^2/sec
    u = np.minimum(t1, t2)
    v = np.maximum(t1, t2)
    return M * u

print 'Uncertainty (Hz)\n'

for ii in range(len(run_list)):
    run_pair = run_list[ii]
    run = run_pair[0:2]
    pair = run_pair[-1]

    filename = file_dict[run]
    key = key_dict[run][pair]

    gold_subruns = np.loadtxt(filename[0])

    subrun_df = pd.read_hdf(filename[1], key='subrun_df')

    # vtm_df = pd.read_hdf(filename[2], key='run_'+run+pair)

    ## calculate the (rough) times of trolley runs from interp dataframes
    tr_interp_df_1 = pd.read_hdf(filename[1], key=key[0])
    t_start = np.mean(tr_interp_df_1.index.values)

    if (run == '1d') & (pair == '6'):
        t_end = 1530261096.0
    else:
        tr_interp_df_2 = pd.read_hdf(filename[1], key=key[1])
        t_end = np.mean(tr_interp_df_2.index.values)

    gold_subruns_df = pd.DataFrame(gold_subruns.astype(int), columns=['run', 'subrun']).merge(subrun_df, on=['run', 'subrun'])

    gold_subruns_df = gold_subruns_df[(gold_subruns_df['start_gps']>=t_start) & (gold_subruns_df['end_gps']<=t_end)]
    
    ## a vector is number of ctags in each subrun divided by the number of ctags in all used subruns
    a = gold_subruns_df['ctags'].values/float(gold_subruns_df['ctags'].sum())

    ## make vector of subrun times after trolley run (use avg)
    t_subrun = ((gold_subruns_df['start_gps']+gold_subruns_df['end_gps'])/2).values - t_start
    t_period = t_end - t_start


    ## build the covariance matrix sigma
    ## t1, t2 are the two times under consideration, can be matrices


    ## make t1 a matrix of proper size with time by row; t2 is by column; ~10 sec
    t1 = np.outer(t_subrun, np.ones(t_subrun.size))
    t2 = np.outer(np.ones(t_subrun.size), t_subrun)

    ## run the cov function (note, this takes ~1 min)
    # sigma = cov(t1, t2, t_period)
    
    if (run == '1d') & (pair == '6'):
        sigma = cov2(t1, t2, t_period, M=1)  # use M=1, scale with rates later to save time
    else:
        sigma = cov(t1, t2, t_period, M=1)
    sigma_avg = np.dot(a, np.dot(sigma, a))
    for m in range(9):
        ## this is the uncertainty of the average (variance) in Hz^2

        print str((sigma_avg*rates[m])**0.5)+'\t',

    print


Uncertainty (Hz)

2.629352900037709	0.4508545333592144	0.3982496108886861	0.2646018589763777	0.8363861166737168	0.11878705341666614	0.2583495659366613	0.19388247896111221	0.27827423960274295	
2.451712228542833	0.4203945285225524	0.3713436264162995	0.24672519741958096	0.7798793649962299	0.11076172827543779	0.24089531307743495	0.18078366147136057	0.25947386374522746	
2.4975376296375957	0.42825219944465714	0.3782844779674722	0.25133678315150704	0.7944562326605569	0.1128319959704306	0.24539793137623217	0.18416272190996216	0.2643237371281244	
2.5376363808476636	0.43512792303613207	0.3843579540939757	0.2553720741589116	0.8072114770431473	0.1146435491103167	0.24933787225277812	0.18711951226234938	0.26856753776129205	
2.7103880363960653	0.4647496094397295	0.410523433669403	0.2727567353045107	0.862163053277365	0.12244800173253839	0.26631175020774156	0.19985782487983028	0.28685049079781944	
2.3971954377007596	0.41104654701159116	0.36308635112265436	0.2412389638287868	0.7625378027490088	0.1082988

In [ ]:
print np.max(np.diagonal(sigma)**0.5*rates[0]**0.5*1000/61.79)

plt.plot(t_subrun, np.diagonal(sigma)**0.5*rates[0]**0.5*1000/61.79, '.')
plt.xlim(0, 74*3600)

In [ ]:
print (np.max(sigma)*rates[0])**0.5 *np.sqrt(72)

In [ ]:
print np.max(np.diagonal(sigma)**0.5*rates[0]**0.5*1000/61.79)

plt.plot(t_subrun, np.sqrt(np.diagonal(sigma)*rates[0])*1000/61.79, '.')
plt.xlim(0, 74*3600)

In [ ]:
%%time

fig, ax = plt.subplots(1,1)

ax.plot(gold_subruns_df['start_gps'].values[0], [1], '.', color='C2', label=r'$\omega_a$ gold runs')
ax.plot(gold_subruns_df['start_gps'], np.ones(gold_subruns_df.shape), '.', color='C2')

# ax.plot(vtm_df.index.values, np.ones(vtm_df.shape[0]), color='C0', label='fixed probe data')
ax.plot([t_start, t_end], [1,1], '.', color='C1', label='trolley run')

ax.legend()
plt2.plt_unix_time_to_CST(ax)
ax.get_yaxis().set_visible(False)

fig.set_size_inches(8,6)
fig.tight_layout()



####


# print 'Period (hr)'
# print t_period/3600

# print 'RW uncertainty (ppb)'
# print 0.01810259*np.sqrt(t_period)*16/np.sqrt(3)
# print ''

# print 'BB uncertainty (ppb)'
# print 0.01810259*np.sqrt(t_period)*16/np.sqrt(12)
# print ''

# print 'Uncertainty (Hz)'
# print sigma_avg**0.5
# print '(ppb)'
# print sigma_avg**0.5*16

In [ ]:
np.sqrt(np.dot(a, np.dot(np.diag(np.diagonal(sigma)), a))*rates[0])

In [ ]:
np.sqrt(np.dot(a, np.dot(sigma, a))*rates[0])